# Example using sklearn pipelines with ta library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler

from ta import *

In [2]:
# SETTINGS

LABELING = 'regression'
target = 'target' # name target column

if LABELING == 'regression':
    score = "neg_mean_absolute_error"
elif LABELING == 'binary':
    score = "accuracy"

In [3]:
# Load data
df = pd.read_csv('../data/datas.csv', sep=',')

# Labeling        
if LABELING == "regression":
    df[target] = (df['Close'] / df['Close'].shift(1)) - 1

elif LABELING == "binary":
    df[target] = 0 # 'KEEP & DOWN'
    df.loc[df.Close > df.Close.shift(1), target] = 1 # 'UP'

# Nans values
df = utils.dropna(df)
    
cols = [i for i in df.columns if i != target]
X = df[cols]
y = df[target]

N = 500

X_train = df[cols][:-N]
y_train = df.target[:-N]
X_test = df[cols][-N:]
y_test = df.target[-N:]

In [4]:
pipe = make_pipeline(
    TAFeaturesTransform("Open", "High", "Low", "Close", "Volume_BTC", fillna=True),
    StandardScaler(),
    ExtraTreesRegressor()
)

In [5]:
y_pred = pipe.fit(X_train, y_train).predict(X_test)

if LABELING == "regression":
    print(mean_absolute_error(y_test, y_pred))
elif LABELING == 'binary':
    print(accuracy_score(y_test, y_pred))

/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/trend.py:634: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  aroon_up = close.rolling(n).apply(lambda x: float(np.argmax(x) + 1) / n * 100)
/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/trend.py:656: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the futu

0.0005068263451268888


In [6]:
resultados = cross_val_score(pipe,
                            X = df[cols],
                            y = df[target],
                            scoring=score)
print(resultados)

/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/wrapper.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[volume], fillna=fillna)
/Users/lecrintech/projects/python/ta/env_ta/lib/python2.7/site-packages/ta/wrapper.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

[-0.10294477 -0.00205641 -0.00436494]
